# Kaggle竞赛入门 - Titanic实战


> “泰坦尼克号”沉没的时候，船长下了一个命令：“让女人和小孩先走！”

我用Titanic竞赛的例子来介绍完整的参加一次kaggle竞赛的基本流程，这篇文章可以希望帮助你解决这几个问题：

- 为什么要参加Kaggle竞赛？
    - 通过kaggle竞赛，入门机器学习。
    - 跟踪机器学习的发展方向
    - 。。。。    
- 我是小白，我要开始参加哪个比赛？
    - 当然是Titanic了，不然我写这文章干嘛。
- 参加Kaggle比赛的基本流程是什么？
    - 了解竞赛的基本信息。
    - Data Explore Analysis（DEA）：下载数据，探索数据
    - Data Clearning:清洗数据
    - Feature engineering：特征工程
    - Model turning：模型训练和模型评估
    - 终极杀器：Model ensemble，模型融合

本篇文章的目的是帮助大家入门Kaggle平台，所以不会有很高的分数，主要以介绍基本流程为主，以完成Titanic竞赛的baseline为目标。

下一篇文章才会教大家如何在baseline的基础上去提高竞赛的成绩。

具体的操作还有问题的请看看我之前的微信公众号文章。

- Kaggle竞赛入门：（一）机器学习环境搭建 https://mp.weixin.qq.com/s?__biz=MzAxMTU3NTkzOQ==&mid=2662346159&idx=1&sn=8552a2228e7ef5defc95129c7a2d3245&chksm=80fa7a43b78df3552d29dab8d6c17b99d50eba2be7dabf7a54b257a4dc75ae9f22058d7f3a3f#rd
- kaggle平台入门（二）Titanic初试身手 https://mp.weixin.qq.com/s?__biz=MzAxMTU3NTkzOQ==&mid=2662346192&idx=1&sn=d662c08f7d356030dd5a3ab67b8a82d4&chksm=80fa7a3cb78df32a5b6e5c120624f1dec3c752568decce55169b61939a25d1209393f01ae3f8#rd

### 导入需要用的包

Python数据分析中，最为常用的包是Numpy,pandas,matplotlib,sciket-learn.

Numpy主要是用于矩阵运算。

Pandas用于数据处理和数据分析。

matplotlib是Python下的最强大的数据可视化工具。

sciket-learn是目前Python机器学习中运用最多的库，他包含了我们在机器学习中常用的模型。本文我引入了tree下的决策树模型。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

### 数据探索

开始一个竞赛之前，我们必须花时间来了解一下我们的比赛数据。

首先，Kaggle的大部分比赛提供的文件都是csv格式的，我们用pandas.read_csv()这个方法来读取数据。

In [25]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
submission = pd.read_csv('../input/gender_submission.csv')

看看我们的数据都有什么？

pandas提供了几个方法，一个是pandas.head()，用于查看数据的前几行数据。

我们调用这个方法后，就可以看到我们数据的前5行。

我解释一下比较重要的数据：
- PassengerId  # 乘客id
- Survived     # 是否获救，1存活，0未存活
- Pclass       # 船舱的等级，分为1，2，3个不同的舱位
- Name         # 乘客姓名
- Sex          # 性别，注意妇女先走
- Age          # 年龄，注意小孩先走
- SibSp        # 
- Parch
- Ticket
- Fare         # 船票价格 
- Cabin
- Embarked     

其实这些描述在比赛的说明里都有的，只是需要花点时间认真读一下。

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


另外一个比较重要的方法就是.describe(),这个方法会快速帮我们计算一些特征的统计指标。

In [5]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,29.599282,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.703770,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


Titanic就是要求我们根据提供的训练数据，预测test数据中的乘客是否幸存。

其实在一个kaggle竞赛中，我们是需要花很多时间来对竞赛的具体任务深入的分析。这些内容希望教给大家，花一点时间，认真的看看Tantinic这个竞赛的描述。

### 数据清洗

数据清洗就是需要我们在数据探索的基础上，对数据进行清洗。比如：异常值处理，缺失值处理，数据归一化等。

Titanic这个比赛，比较重要的几个特征是年龄、性别，因为船长说了女人和小孩先走，所以女性和年龄小的幸存的概率更大。

为了让大家快速上手，我不做过多的特征处理，主要是采用数值型特征，包括：Pclass，Age，SibSp，Parch，Fare。

In [14]:
# Age 的count这里只有714行，而其他的列都有891行，说明Age有缺失值，我们用中位数来填充。
train["Age"] = train["Age"].fillna(train["Age"].median())
# 对test进行同样的操作
test["Age"] = test["Age"].fillna(test["Age"].median())

# test的fare也有一个缺失值，同样的处理办法
train["Fare"] = train["Fare"].fillna(train["Fare"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

### 特征工程

特征工程是机器学习中最重要的一个环节之一，好的特征工程可以显著我们的模型性能。

In [15]:
# 我选择最简单的几个特征
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# train，test里对应的Pclass，Age，SibSp，Parch，Fare作为训练数据和测试数据。
x_train = train[features]
x_test = test[features]
# train的Survived字段为我们的训练标记。
y_train = train['Survived']

看一下我们的训练数据和测试数据。

In [16]:
x_train.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [17]:
x_test.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,34.5,0,0,7.8292
1,3,47.0,1,0,7.0000
2,2,62.0,0,0,9.6875
3,3,27.0,0,0,8.6625
4,3,22.0,1,1,12.2875


### 模型训练

准备好训练数据之后，就需要用模型进行训练。

Titanic是要求我们判断乘客是否幸存，就是简单的二分类问题，将数据分为两类（0、1）。

对于二分类问题，我们可以采用比较简单的决策树模型的处理。

In [18]:
# 创建决策树model
model_dt = DecisionTreeClassifier()
# 利用训练数据训练模型，x_train是训练数据，y_trian是训练标记
model_dt.fit(x_train, y_train)
# 利用训练好的模型在测试集上进行预测
y_prediction = model_dt.predict(x_test)

看一下我们的预测结果。

In [22]:
# 查看前20个预测结果
y_prediction[:20]

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0])

### 模型评估

我们的模型训练好了，但是现在你知道模型的性能怎么样吗？

这里就需要我们队模型进行评估。在真实的应用场景中，我们需要将数据分为3部份，比例为8：1：1，8为train，1为valid，1为test。

这部分的内容我以后再完善。

Kaggle比赛中，我们有一部分测试数据是放在Kaggle上的，Kaggle利用这部分数据对我们的模型进行评估，我们在Kaggle上的分数就是这么来的。

### 提交结果

将我们的预测结果保存到model-dt-submission.csv文件中，并提交。

In [26]:
submission['Survived'] = y_prediction
submission.to_csv('model-dt-submission.csv',index=False,encoding="utf-8")

提交结果。


<img src="../images/submit.png">

成绩非常不理想，问题出在哪里呢？留下来给大家思考。

本篇文章，我用了Titanic的数据集，写了一个baseline，下一篇文章，我会从机器学习的每个环节入手，手把手教大家把成绩提高到Top10%。

## 参考资料

- 